# Approaching (Almost) Any NLP Problem on Kaggle

우선 이 노트북은 [Abhishek Thakur의 노트북](https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle/notebook)을 필사하였고 한국어로 번역하였습니다.

이 글에서 저는 캐글의 NLP문제의 접근법에 대해 얘기할 예정입니다.  
예시로 저는 이 대회의 데이터를 사용하겠습니다.  
처음으로 아주 간단한 첫 모델을 만들 것이고 그것을 다른 feature들을 통해 향상시킬 것입니다.  
또한 어떻게 심층 신경망이 사용되는가와 이 글의 마지막에서는 일반적으로 여러 아이디어를 앙상블하는 것을 다루겠습니다.

Spooky NLP tutorial 2 - Abhishek Thakur Kor.ver### This covers:

* tfidf
* count features
* logistic regressing
* naive bayes
* svm
* xgboost
* grid search
* word vectors
* LSTM
* GRU
* Ensembling

*NOTE*: 이 노트북은 이 데이터 셋에서 높은 점수를 리더보드에 기록하는 것이 목적이 아닙니다.  
하지만 적절하게 모든 과정들을 따라오신다면, 여러분은 일부 수정작업을 통해 높은 점수를 기록할 수 있을겁니다. ;)


이제 각설하고, 제가 사용할 중요한 파이썬 모듈을 임포트하고 시작해봅시다!

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

이제 데이터셋을 불러옵시다.

In [ ]:
train = pd.read_csv('../input/spooky/train.csv')
test = pd.read_csv('../input/spooky/test.csv')
sample = pd.read_csv('../input/spooky/sample_submission.csv')

이번에는 데이터를 빠르게 확인해봅시다.

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample.head()

이 대회는 텍스트가 주어진 저자인 EAP, HPL, MWS를 예측하는 것이 목적입니다.  
간단하게 말하면 서로 다른 3개의 클래스를 가진 텍스트 분류 문제입니다.  


이 특정 문제에 대해, 캐글은 multi-calss log-loss 평가 지표를 사용합니다.  
이것은 아래의 링크로 구현이 되어 있습니다.  
[https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py](https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py)  

$$ L_{\log}(y, p) = -(y \log (p) + (1 - y) \log (1 - p)) $$

In [ ]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

scikit-learn의 LabelEncoder를 활용하여 텍스트 라벨을 0, 1, 2의 정수로 변환하겠습니다.

In [ ]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

더 진행하기 전에 데이터셋을 훈련집합과 검증집합으로 분리하는 것이 중요합니다.  
scikit-learn모듈에서 `model_selection`의 `train_test_split`을 사용하면 됩니다.

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [ ]:
print(xtrain.shape)
print(xvalid.shape)

## Building Basic Models

이제 첫번째 모델을 만들어봅시다!  


첫모델은 아주 간단한 로지스틱 회귀에 기반한 TF-IDF (Term Frequency - Inverse Document Frequency)입니다.

In [ ]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [ ]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

첫번째 모델의 multiclass logloss값은 0.626를 기록했습니다.  


하지만 저는 좀 더 좋은 점수가 욕심이 납니다. 그래서 다른 데이터를 사용해서 동일한 모델을 보겠습니다.  

TF-IDF를 사용하는 대신 단어 수를 feature로 사용할 수 있습니다. 이 방법은 scikit-learn의 CountVectorizer을 활용하면 쉽게 사용할 수 있습니다.

In [ ]:
ctv = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}',
                     ngram_range=(1, 3), stop_words='english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

In [ ]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

와~! 첫 모델보다 0.1점을 향상시켰습니다!!  

다음으로, 예전에 꽤 유명했던 모델인 Naive Bayes 모델을 사용해보겠습니다.  

두 데이터셋에 대해서 naive bayes가 어떤 결과를 보여주는지 한번 봅시다!

In [ ]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

괜찮은 결과입니다!  
하지만 횟수를 사용한 로지스틱 회귀가 좀 더 낫습니다!  
횟수를 사용한 데이터를 사용하면 어떤 일이 벌어질까요?

In [ ]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

와! 오래된 방식이 여전히 잘 동작하는 것 같네요!  
또 다른 예전 방식에는 SVM이 있습니다. 어떤 사람들은 SVM을 사랑할 정도로 정말 좋아합니다.  
이제 한번 SVM을 데이터셋에 적용시켜봅시다.  

SVM은 많은 시간이 걸리므로 SVM을 적용하기 전에 Singular Value Decomposition(특잇값 분해)을 활용하여 TF-IDF의 기능 수를 줄일 것입니다.  

또한, SVM을 적용하기 전에 데이터를 반드시 표준화해야 합니다.

In [ ]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

이제 SVM을 적용해봅시다. 다음 셀을 실행하고 여자친구나 남자친구랑 잠깐 밖에 산책하면서 자유를 느끼세요 :P

In [ ]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

시간이 다 되었네요! SVM은 이 데이터에 대해서 좋은 성능을 내지는 못하는 걸로 보입니다...  

다음으로 넘어가기 전에, 캐글에서 유명한 알고리즘인 xgboost를 한번 봅시다!

In [ ]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learnig_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

In [ ]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learnig_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

In [ ]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learnig_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

In [ ]:
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print('logloss: %0.3f ' %multiclass_logloss(yvalid, predictions))

XGBoost는 별로 운이 없는 거 같네요! 하지만 그게 항상 맞는 말인건 아닙니다.  
아직 하이퍼파라미터 최적화를 진행하지 않았기 때문입니다. 저는 게으르기 때문에 그걸 어떻게 하는지를 알려드리고 여러분 맘대로 하셔도 괜찮습니다! ;)  
이건 다음 섹션에서 얘기하겠습니다.

## Grid Search

하이퍼파라미터 최적화를 위한 기술입니다. 그렇게 효과적이진 않지만 사용하고자 하는 그리드를 알고 있다면 좋은 결과를 가져올 수 있습니다.  
저는 이 글에서 일반적으로 사용해야하는 매개변수를 특정합니다:  
[http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/](http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/)  
이 파라미터들은 제가 자주 사용하는 파라미터들입니다.  
효과적일 수도 있고 아닐 수도 있는 다른 하이퍼파라미터 최적화 방법들이 있습니다.  


이 섹션에서는 저는 로지스틱 회귀를 활용한 그리드 서치에 대해 말씀드리겠습니다.  


그리드 서치를 진행하기에 앞서 저는 scoring함수를 만들겠습니다. 이 함수는 scikit-learn의 `make_scorer`를 활용하여 만들어집니다.

In [ ]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

다음으로 저는 파이프라인이 필요합니다.  
여기에선 임시적으로 파이프라인에 SVD, scaling 그리고 로지스틱 회귀를 사용하겠습니다.  
파이프라인에는 1개의 모듈만 있는 것보단 여러개가 있는 것이 좋습니다.

In [ ]:
# Initialize SVD
svd = TruncatedSVD()

# Initialize the standard scaler
scl = preprocessing.StandardScaler()

# We will use logistic regression here.
lr_model = LogisticRegression()

# Create the pipeline
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

다음으로는 파라미터의 그리드가 필요합니다:

In [ ]:
param_grid = {'svd__n_components': [120, 180],
             'lr__C': [0.1, 1.0, 10],
             'lr__penalty': ['l1', 'l2']}

SVD의 경우 120에서 180개의 요소를 평가하고 로지스틱 회귀의 경우 l1과 l2 패널티로 C값의 3가지 다른 값을 평가할 것입니다.  
이제 이 매개변수들에서 그리드 서치를 할 수 있습니다.

In [ ]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                    verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain) # we can use the full data here but im only using xtrain
print('Best score: %0.3f' %model.best_score_)
print('Best parameters set:')
best_parameters = model.best_estimator_.get_params()

for param_name in sorted(param_grid.keys()):
    print('\t%s: %r' %(param_name, best_parameters[param_name]))

점수가 제가 가진 SVM과 유사하게 나타났습니다.  
이 기술은 xgboost나 다항 naive bayes를 미세 조정하는 데에 사용할 수 있습니다.  
여기서는 tfidf 데이터를 사용합니다.

In [ ]:
nb_model = MultinomialNB()

# Create the pipeline
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                    verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain) # we can use the full data here but im only using xtrain
print('Best score: %0.3f' %model.best_score_)
print('Best parameters set:')
best_parameters = model.best_estimator_.get_params()

for param_name in sorted(param_grid.keys()):
    print('\t%s: %r' %(param_name, best_parameters[param_name]))

이 방식은 기존의 naive bayes score에서 8%정도 향상을 했습니다!  

NLP문제에서 단어 벡터를 보는 것이 일반적이비다. Word vector들은 데이터에 대한 많은 통찰력을 제공합니다.  
그 내용들을 한번 알아봅시다.

## Word Vectors

너무 많은 디테일은 빼고 저는 어떻게 문장 벡터를 만드는지와 어떻게 그것들을 사용해서 위에서 같이 학습 모델을 만들 수 있는지를 설명하겠습니다.  
저는 GloVe 벡터, word2vec 그리고 fasttext의 팬입니다. 저는 이번에는 GloVe 벡터를 사용하겠습니다.  
GloVe 벡터는 여기에서 다운받을 수 있습니다.  
[http://www-nlp.stanford.edu/data/glove.840B.300d.zip](http://www-nlp.stanford.edu/data/glove.840B.300d.zip)

In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}

f = open('../input/glove840b300dtxt/glove.840B.300d.txt')

for line in tqdm(f):
    values = line.split()
    word = values[0]
    
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except:
        continue
    
    embeddings_index[word] = coefs
    
f.close()

print(f'Found {len(embeddings_index)} word vectors.')

In [ ]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]

    M = []
    
    for w in words:
        try:
            M.append(embedding_index[w])
        except:
            continue
    
    M = np.array(M)
    v = M.sum(axis=0)
    
    if type(v) != np.ndarray:
        return np.zeros(300)
    
    return v / np.sqrt((v ** 2).sum())

In [ ]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]

In [ ]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

이제 glove feature를 사용한 xgboost의 성능을 봅시다:

In [ ]:
# Fitting a simple xgboost on globe features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print('logloss: %.3f ' %multiclass_logloss(yvalid, predictions))

이제 매개변수의 간단한 조정이 GloVe로 xgboost 점수를 향상시키는 것을 알 수 있습니다.

## Deep Learning

하지만 이 파트는 딥러닝입니다! 우리는 신경망의 훈련없이는 살 수 없습니다.  
그래서 저는 LSTM을 훈련하고 간단한 선형 네트워크에 GloVe를 적용할 것입니다.  
일단 선형 네트워크부터 해봅시다!

In [ ]:
# scale the data vefore any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [ ]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [ ]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64,
         epochs=5, verbose=1,
         validation_data=(xvalid_glove_scl, yvalid_enc))

더 좋은 결과를 얻기위해 신경망의 매개변수들을 조정하고, 더 많은 층을 추가하고, dropout비율을 올릴 필요가 있습니다.  
제가 어떤 조정도 없이 xgboost보다 더 좋은 결과를 빠르게 얻는 방법을 보여드리겠습니다.  

LSTM과 텍스트를 토큰화 합니다.

In [ ]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))

for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                   300,
                   weights=[embedding_matrix],
                   input_length=max_len,
                   trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

이제 점수가 0.5보다 아래로 내려간 것을 볼 수 있습니다.  
멈춤 없이 많은 수의 에포크를 돌렸는데요, 하지만 조기멈춤을 사용하면 최적의 반복으로 값을 찾을 수 있습니다.  
그렇다면 조기멈춤은 어떻게 할까요?  


아주 쉽습니다. 모델을 다시 컴파일 해볼까요?

In [ ]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                   300,
                   weights=[embedding_matrix],
                   input_length=max_len,
                   trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

한가지 의문점이 생깁니다: 왜 드롭아웃을 썼을까요?  
만약, 드롭아웃이 적거나 없다면 모델을 학습할 때, 오버피팅이 발생할 수 있습니다. :)  

이제 양방향 LSTM이 더 좋은 결과를 주는지 확인해 봅시다. 케라스를 쓴다면 식은 죽 먹기입니다:)

In [ ]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                   300,
                   weights=[embedding_matrix],
                   input_length=max_len,
                   trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

꽤 가까워졌네요! 이제 2개층의 GRU를 써봅시다.

In [ ]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                   300,
                   weights=[embedding_matrix],
                   input_length=max_len,
                   trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100,
         verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

좋습니다! 이전보다 훨씬 더 좋아졌습니다! 계속해서 조정을 하고 성능을 향상하겠습니다.  
시도해 볼 가치로는 형태소 분석과 표제어 추출이 있습니다.  
일단 저는 이 방법은 현재 넘어가겠습니다.  


캐글의 세계에서는 높은 점수를 얻기 위해서는 앙상블 모델을 사용하는 것이 좋습니다.  
이제 약간의 앙상블을 해봅시다!

## Ensembling

몇 달 전 저는 간단한 앙상블러를 만들었습니다. 하지만 완전히 개발할 시간이 없었습니다.  
그때 만든 모델을 사용하겠습니다.

In [ ]:
# this is the main ensembling class. how to use it is in the next cell!
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import os
import sys
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format="[%(asctime)s] %(levelname)s %(message)s",
    datefmt="%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger(__name__)


class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification', optimize=roc_auc_score,
                 lower_is_better=False, save_path=None):
        """
        Ensembler init function
        :param model_dict: model dictionary, see README for its format
        :param num_folds: the number of folds for ensembling
        :param task_type: classification or regression
        :param optimize: the function to optimize for, e.g. AUC, logloss, etc. Must have two arguments y_test and y_pred
        :param lower_is_better: is lower value of optimization function better or higher
        :param save_path: path to which model pickles will be dumped to along with generated predictions, or None
        """

        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path

        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None

    def fit(self, training_data, y, lentrain):
        """
        :param training_data: training data in tabular format
        :param y: binary, multi-class or regression
        :return: chain of models to be used in prediction
        """

        self.training_data = training_data
        self.y = y

        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
            logger.info("Found %d classes", self.num_classes)
            self.lbl_enc = LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)

        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((train_prediction_shape[0],
                                                          train_prediction_shape[1] * len(self.model_dict[level])))

        for level in range(self.levels):

            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
                    logger.info("Training Level %d Fold # %d. Model # %d", level, foldnum, model_num)

                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])

                    logger.info("Predicting Level %d. Fold # %d. Model # %d", level, foldnum, model_num)

                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index,
                        (model_num * self.num_classes):(model_num * self.num_classes) +
                                                       self.num_classes] = temp_train_predictions

                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
                    logger.info("Level %d. Fold # %d. Model # %d. Validation Score = %f", level, foldnum, model_num,
                                validation_score)
                    foldnum += 1
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
                logger.info("Level %d. Model # %d. Mean Score = %f. Std Dev = %f", level, model_num,
                            avg_score, std_score)

            logger.info("Saving predictions for level # %d", level)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, "train_predictions_level_" + str(level) + ".csv"),
                                        index=False, header=None)

        return self.train_prediction_dict

    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((test_prediction_shape[0],
                                                         test_prediction_shape[1] * len(self.model_dict[level])))
        self.test_data = test_data
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level - 1]
                temp_test = self.test_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):

                logger.info("Training Fulldata Level %d. Model # %d", level, model_num)
                if level == 0:
                    model.fit(temp_train[0][model_num], self.y_enc)
                else:
                    model.fit(temp_train, self.y_enc)

                logger.info("Predicting Test Level %d. Model # %d", level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num * self.num_classes): (model_num * self.num_classes) +
                                                                                        self.num_classes] = temp_test_predictions

                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, "test_predictions_level_" + str(level) + ".csv"),
                                       index=False, header=None)

        return self.test_prediction_dict

In [ ]:
# specify the data to be used for every level of ensembling:
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')

ens.fit(train_data_dict, ytrain, lentrain=xtrain_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=xvalid_glove.shape[0])

In [ ]:
# check error
multiclass_logloss(yvalid, preds[1])

게다가, 우리는 앙상블이 점수를 크게 향상시키는 것을 볼 수 있습니다!  
이건 듀토리얼이므로 따로 제출 csv는 제공하지 않습니다!